In [1]:
import os
import re
import sys
from textblob2 import load, save
from itertools import chain, combinations
import random
import requests as r
import json
from pprint import pprint
from datetime import datetime
import time
import math
import copy
from bs4 import BeautifulSoup as bs
import langid
import csv
import re
import py8chan as pc
from pathlib import Path
from pattern.text import parse, language, find_tokens
from pattern.text.fr import tokenize as tokenize_fr
from pattern.text.fr import tag as tag_fr
from collections import Counter, defaultdict
import grasp.grasp as grasp

In [2]:
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0"}

ANDROID = "android_offensive_words.txt"
CRUCI_GR = "cruciverbistes_grossièreté.txt"
CRUCI_NATIO = "cruciverbistes_insultes_natio.txt"
CRUCI_GEN = "crucivervistes_inusltes_general.txt"
WIKI = "wiki_insultes.txt"
INSULT_FILES = [ANDROID,CRUCI_GR,CRUCI_NATIO,CRUCI_GEN,WIKI]

SMS88 = "88milSMS.csv"

<font size = 22>#########################</font>

<font size = 5>8chan Scraping </font>

In [3]:
FOLDER8 = r"C:\Users\Pierre\Documents\@ADMIN\#KUL - IA\_Stage\TexTGaiN\GSoC\French POW List\8chan_fr"

CHAN8 = "https://8ch.net/{}/index.html"
# DEMPART_URL_RE = re.compile(r"https://8ch.net/dempart/res/\d{6}")
# DEMPART_URL = "https://8ch.net/dempart/2.html"
SEARCH8 = "https://8ch.net/search.php?search={}&board={}"

CATALOG8 = "https://8ch.net/{}/catalog.html"
ARCHIVE8 = "https://8ch.net/{}/archive/index.html"
THREAD_RE = re.compile(r"\"id\":\"<a href=\\\"/\w+/res/(\d+)\.html\\\"")

OP = re.compile(r"op_\d{6}")
REPLY = re.compile(r"reply_\d{6}")
GREEN_QUOTE = re.compile(r">.*$")

CSV_HEADER = ['id',"type", "text"]


with open("8ch_all_boards.txt", "r") as f:
    BOARDS = f.read().split("\n")
with open("fr_boards_with_langid.txt", "r") as f:
    FR_BOARDS = f.read().split("\n")
    
CH8_PREPROCESSED = r"C:\Users\Pierre\Documents\@ADMIN\#KUL - IA\_Stage\TexTGaiN\GSoC\French POW List\8chan_fr\all_boards_preprocessed.csv"

In [4]:
def has_results(bs_response):
    """used to check whether a search with SEARCH8 yields results or not"""
    res = bs_response.find("legend")
    if res:
        return True
    else:
        return False
    

<font size = 22>#########################</font>

JeuxVideos.com Scraping

In [5]:
JVBASE = "http://www.jeuxvideo.com/"
JVSEARCH = "http://www.jeuxvideo.com/recherche/forums/0-51-0-1-0-{}-0-blabla-18-25-ans.htm?search_in_forum={}&type_search_in_forum=texte_message"
JVFOLDER = r"C:\Users\Pierre\Documents\@ADMIN\#KUL - IA\_Stage\TexTGaiN\GSoC\French POW List\JV"
JVPOL = "http://www.jeuxvideo.com/forums/0-55-0-1-0-{}-0-politique.htm"

JV_POL, JV_18 = r"JV\jv_messages.csv", r"JV\jv_messages_18_25.csv"
JV_CLEAN = r"JV\jv_pol_18_clean.csv"

with open(os.path.join(JVFOLDER, "_triggers.txt")) as f:
    TRIGGERS = f.read().split("\n")
POST_LIMIT = 200
TOPIC_HEADER = ['id', 'url']
MESSAGE_HEADER = ['id', 'parent_id', 'text', 'date']
DATE_RE = re.compile(r"\d{2}\s\w+\s\d{4}")

In [6]:
def has_next_page(search_results):
    """
    :search_results: is a BeautifulSoup object
    """
    page_after = search_results.find("div", attrs={"class":"pagi-after"})
    next_ = page_after.find("a")
    if next_:
        return next_["href"]
    else:
        return ""

In [7]:
def get_all_topics(page):
    topics_d = []
    topic_list = page.find("ul", attrs={"class":"topic-list topic-list-admin"})
    topics = topic_list.findAll("li", attrs={"class":""})
    
    for t in topics:
        nbr_posts = t.find("span", attrs={"class":"topic-count"})
        nbr_posts = int(nbr_posts.text)
        if nbr_posts <= POST_LIMIT:
            id_ = t["data-id"]
            url = t.find("a")["href"]
            topics_d.append({"id":id_,"url":url})
        else:
            continue
    return topics_d

In [8]:
def write_topic_ids(new_ids):
    new_ids = set(new_ids)
    with open(os.path.join(JVFOLDER, COLLECTED_TOPICS), "r") as f:
        ids = f.read().split("\n")
        ids = set(ids)
    ids = ids.union(new_ids)
    with open(os.path.join(JVFOLDER, COLLECTED_TOPICS), "w") as f:
        f.write("\n".join(ids))

In [9]:
def find_date(message):
    m =  re.search(DATE_RE, message.text).group()
    if m:
        return m
    else:
        return ""

In [10]:
def isolate_id(url):
    return re.sub(r"forums/\d+-\d+-(\d+)-\d+-\d+-\{\}-\d+-\S+", r"\1", url)
     

In [11]:
def remove_html_tag(s):
    a = re.sub(r"<.*>","",s)
    return a

In [12]:
def remove_url(post):
    while re.search(g.URL, post):
        post = re.sub(g.URL, "", post)
    return post

In [13]:
def get_all_messages(page, url):
    messages_raw = page.findAll("div", attrs={"class":"bloc-message-forum"})
    parent_id = isolate_id(url)    
    messages = []    
    for m in messages_raw:
        id_ = m["data-id"]
        
        
        text = m.find("div",attrs={"class":"txt-msg text-enrichi-forum "})
        bqs = text.findAll("blockquote")
        for b in bqs:
            b.extract()
        text = text.text
        
        date = find_date(m)

        messages.append({"id":id_,"parent_id":parent_id,"text":text,"date":date})
    return messages

In [14]:
def read_done():
    with open("done_IDS.txt", "r") as f:
        id_ = f.read()
        id_ = id_.split(sep="\n")
        while "\n" in id_:
            id_.remove("\n")
    return id_

In [15]:
def write_done(id_):
    file = "done_IDS.txt"
    if not os.path.exists(file):
        with open(file, "w") as f:
            f.write(id_)
    else:
        with open(file, "a") as f:
            f.write("\n"+id_)
    

In [16]:
def is_done(id_):
    done = read_done()
    if id_ in done[::-1]:
        return True
    else:
        return False

In [ ]:
def kw_cont_table(kw, tc, rc):
    """
    outputs a contingency table for the keyword "kw", in the target corpus "tc" and the reference corpus "rc"
    """
    row1 = [tc.get(kw, 0), sum(tc.values()) - tc.get(kw, 0)]
    row2 = [rc.get(kw, 0), sum(rc.values()) - rc.get(kw, 0)]
 
    return [row1[0], row1[1], row2[0], row2[1]]

In [ ]:
def exp_freq_table(cont):
    """
    outputs a frequency table for the keyword "kw", for the target corpus "tc" and the reference corpus "rc" (cf. kw_cont_table)
    """ 
    r1 = cont[0] + cont[1]
    r2 = cont[2] + cont[3]
    c1 = cont[0] + cont[2]
    c2 = cont[1] + cont[3]

    n = r1 + r2
    
    a = (r1 * c1)/n
    b = (r1 * c2)/n
    c = (r2 * c1)/n
    d = (r2 * c2)/n

    return [a, b, c, d]

In [ ]:
def kw_pmi(kw, tc, rc, freq_filter = 5):
    """"
    computes pmi value for keywords
    """
    
    cont_table = kw_cont_table(kw, tc, rc)
    freq_table = exp_freq_table(cont_table)

    if cont_table[0] >= freq_filter and kw in rc and kw in tc:
        return math.log2(cont_table[0]/freq_table[0])
    else:
        return 0.0

<font size = 22>#########################</font>

<font size = 22>#########################</font>

<font size = 5>Wiki Scraping </font>

In [90]:
URL1 = "https://fr.wiktionary.org/wiki/Cat%C3%A9gorie:Insultes_en_fran%C3%A7ais"
URL2 = "https://fr.wiktionary.org/w/index.php?title=Cat%C3%A9gorie:Insultes_en_fran%C3%A7ais&pagefrom=lopette#mw-pages"
URL3 = "https://fr.wiktionary.org/wiki/Cat%C3%A9gorie:Insultes_en_fran%C3%A7ais_qu%C3%A9b%C3%A9cois"

In [93]:
all_w = []
for url in [URL1,URL2,URL3]:
    u = r.get(url, headers=HEADERS)
    u = bs(u.text)
    groups = u.findAll("div", attrs={"class":"mw-category-group"})
    for g in groups:
        ws = g.findAll("a")
        all_w.extend([e.text for e in ws])
all_w = set(all_w)

In [94]:
with open("wiki_insultes.txt","w", encoding="utf8") as f:
    l = "\n".join(all_w)
    f.write(l)

<font size = 22>#########################</font>

<font size = 5>8chan Scraping </font>

vvvvvv Finding FR boards on 8chan using language detection on previously flagged boards vvvvvv

In [166]:
fr_ids = {b:set() for b in FR_BOARDS}
for fr_board in FR_BOARDS:
    response = r.get(ARCHIVE8.format(fr_board), headers=HEADERS)
    if response:
        archive_ids = re.findall(THREAD_RE,response.text)
        archive_ids = {re.search(r"\d+",e).group() for e in archive_ids}
        fr_ids[fr_board] = fr_ids[fr_board].union(archive_ids)
    catalog = pc.Board(fr_board)
    catalog_ids = catalog.get_all_thread_ids()
    catalog_ids = [str(e) for e in catalog_ids]
    catalog_ids = set(catalog_ids)
    fr_ids[fr_board] = fr_ids[fr_board].union(catalog_ids)
for board,ids in fr_ids.items():
    with open(os.path.join(FOLDER8, board)+"_ids.txt", "w") as f:
        f.write("\n".join(ids))

In [178]:
fr_ids = dict()
for file in os.listdir(FOLDER8):
    with open(os.path.join(FOLDER8,file)) as f:
        boardname = file.replace("_ids.txt", "")
        ids = f.read().split("\n")
        ids = [int(e) for e in ids]
        fr_ids[boardname] = ids

In [205]:
for board, ids in fr_ids.items():
    b = pc.Board(board)
    with open(os.path.join(FOLDER8, board+".csv"),"w", encoding="utf8", newline='') as f:
        w = csv.writer(f)
        w.writerow(CSV_HEADER)
        for i in ids:
            posts = []
            t = b.get_thread(i)
            if t:
                ps = t.all_posts
                for p in ps:
                    id_ = p.post_id
                    if p.is_op:
                        type_ = "T"
                    else:
                        type_ = "R"
                    text= p.text_comment
                    posts.append([id_,type_,text])
                w.writerows(posts)

<font size = 22>#########################</font>

FR 8Chan Preprocessing

In [ ]:
all_8chan = []
a = all_8chan
for fr in FR_BOARDS:
    all_8chan.extend(grasp.csv("8chan_fr\\{}.csv".format(fr)))

In [ ]:
a = a[1:]
at = [e[2] for e in a]

In [ ]:
for i,e in enumerate(at):
    e = e.strip()
    e = remove_url(e)
#     e = remove_html_tag(e)
    e = e.lower()
    e = grasp.deflood(e)
    e = " ".join(tokenize_fr(e))
    at[i] = e

<font size = 22>#########################</font>

JeuxVideos.com Scraping

In [56]:
#getting all urls of topics to scrape from the 1st hundred pages
nbr_pages = 100
for i in range(1,(nbr_pages-1)*25, 25) :
    rep = r.get(JVPOL.format(i), headers=HEADERS)
    rep = bs(rep.text)
    topics = get_all_topics(rep)
    with open(os.path.join(JVFOLDER, "topic_ids_url.csv"),'a') as f:
        w = csv.DictWriter(f, TOPIC_HEADER)
        w.writerows(topics)

In [12]:
#loading topics to scrape in memory
with open(os.path.join(JVFOLDER, "topic_ids_url.csv"),'r') as f:
        csvr = csv.DictReader(f, TOPIC_HEADER)
        ids_url = list(csvr)
        urls = [e['url'] for e in ids_url]

In [ ]:
all_urls = len(urls)
current_url = 1
messages = []
for url in urls:
    if not is_done(url):
        if current_url%10 == 0:
            print("{}/{}".format(current_url,all_urls))
        try:
            url = re.sub(r"(\d{2}-\d{2}-\d+-\d-\d-)\d(-\d)", r"\1{}\2", url)
            rep = r.get(JVBASE+url.format(1), headers=HEADERS)
            rep = bs(rep.text)

            nbr_pages = rep.find("div", attrs={"class":"bloc-liste-num-page"})
            nbr_pages = nbr_pages.findAll("span")[-1].text
            nbr_pages = int(nbr_pages)

            messages.extend(get_all_messages(rep, url))
            if nbr_pages > 1:
                for i in range(2,nbr_pages+1):
                    rep = r.get(JVBASE+url.format(i), headers=HEADERS)
                    rep = bs(rep.text)
                    messages.extend(get_all_messages(rep, url))

            file_name = "jv_messages.csv"
            with open(os.path.join(JVFOLDER, file_name),"a",newline="",encoding="utf8") as f:
                dw = csv.DictWriter(f, MESSAGE_HEADER)
                dw.writerows(messages)
            messages.clear()

        except TimeoutError as e:
            print(e)
            current_url += 1
            continue

        except ConnectionError as e:
            print(e)
            current_url += 1
            continue

        except AttributeError as e:
            print(current_url, url, e)
            current_url += 1
            continue
    write_done(url)    
    current_url += 1

<font size = 22>#########################</font>

JVC Preprocessing

In [54]:
posts = []
with open(JV_POL, encoding="utf8", newline="") as f:
    posts.extend(csv.reader(f))
with open(JV_18, encoding="utf8", newline="") as f:
    posts.extend(csv.reader(f))

In [58]:
txt = [e[2] for e in posts]

In [59]:
for i,p in enumerate(txt):
    p = p.strip()
    p = remove_url(p)
    p = p.lower()
    p = " ".join(tokenize_fr(p))
    txt[i] = p
txt = [[e] for e in txt]

In [66]:
with open(JV_CLEAN, "w", encoding="utf8", newline="") as f:
    w = csv.writer(f)
    w.writerows(txt)

In [23]:
final_folder = r"D:\4-8chan Data\preprocessed_48"
files = os.listdir(final_folder)
files = [e for e in files if e.endswith(".csv")]
#4chan: text is 2nd column
#8chan: text is 2nd column
for f in files:
    x = grasp.csv(os.path.join(final_folder, f))
    if f.startswith("4c"):
        x = x[1:]
    for i,e in enumerate(x):
#         print(e)
        e = e[2]
        e = e.split()
        x[i] = [e]
#         print(e)
    y = grasp.csv(os.path.join(test, f))
    y.update(x)
    y.save()
        

In [15]:
test = r"D:\4-8chan Data\preprocessed_48\test"

<font size = 22>#########################</font>

In [17]:
#SMS_preprocessing
s8 = grasp.csv(SMS88, encoding="latin",separator=";")
sms = [e[-1] for e in s8[1:]]
for i,e in enumerate(sms):
    e = e.strip()
    e = remove_url(e)
    e = remove_html_tag(e)
    e = e.lower()
    e = grasp.deflood(e)
    e = " ".join(tokenize_fr(e))
    sms[i] = e
sms = [[e] for e in sms]
sms_clean = grasp.csv("sms_clean.csv", encoding="latin")
sms_clean.update(sms)
sms_clean.save()

<font size = 22>#########################</font>

<font size = 5>Android Sorting </font>

In [ ]:
ANDFILE = "android_fr_wordlist.txt"
with open(ANDFILE, encoding="utf8") as f:
	raw = f.read()

x = raw.split()
x = [e for e in x if "possibly_offensive" in e or re.search(r"flags=\w,", e)]
d = []

for line in x:
    try:
        word = re.search(r"^word=(.*),f=", line)[1]
        print(word)
        d.append(word)
    except TypeError:
        continue
l = "\n".join(d)

with open("android_offensive_words.txt", 'w', encoding="utf8") as f:
    f.write(l)

<font size = 22>#########################</font>

<font size = 5>RECAP - All insults </font>

In [124]:
all_words = []
for file in INSULT_FILES:
    with open(file) as f:
        raw = f.read()
        words = raw.split("\n")
        all_words.extend(words)
all_words = set(all_words)

<font size = 22>#########################</font>

In [17]:
sms = grasp.csv("sms_clean.csv")
jv = grasp.csv(JV_CLEAN)
ch8 = grasp.csv(CH8_PREPROCESSED)

sms= [e[0] for e in sms]
jv= [e[0] for e in jv]
ch8 = [e[0] for e in ch8]

ch8 = [tag_fr(e) for e in ch8]
for i,e in enumerate(ch8):
    e = [w[0] for w in e if w[1].startswith("NN") or w[1].startswith("JJ") or w[1].startswith("VB")]
    ch8[i] = " ".join(e)

sms = [tag_fr(e) for e in sms]
for i,e in enumerate(sms):
    e = [w[0] for w in e if w[1].startswith("NN") or w[1].startswith("JJ") or w[1].startswith("VB")]
    sms[i] = " ".join(e)
    
jv = [tag_fr(e) for e in jv]
for i,e in enumerate(jv):
    e = [w[0] for w in e if w[1].startswith("NN") or w[1].startswith("JJ") or w[1].startswith("VB")]
    jv[i] = " ".join(e)

In [18]:
sms_c, jv_c, ch8_c = Counter(" ".join(sms).split()), Counter(" ".join(jv).split()), Counter(" ".join(ch8).split())

In [ ]:
keys = set(ch8_c.keys()).union(set(sms_c.keys()))
pmi_dict = dict()
for k in keys:
    pmi_dict[k] = own.kw_pmi(k, ch8_c, sms_c)
pmi_list = [e for e in pmi_dict.items()]
pmi_list.sort(key=lambda x:x[1], reverse=True)
print(pmi_list[10:20])

In [20]:
y = own.csv("pmi_list_fr_2.csv")
y.update(pmi_list)
y.save()

In [46]:
div = sum([v for k,v in ch8_c.items()])/1000000

In [80]:
fp  = own.csv(r"C:\Users\Pierre\Downloads\French POW - EN.csv")
fp = fp[2:]


In [ ]:
for i,e in enumerate(fp):
    fp[i][6:] = ["FALSE"]*10

In [85]:
a = own.csv(r"C:\Users\Pierre\Downloads\French POW - EN.csv")
a.clear()
a.update(fp)
a.save()

In [ ]:
for i,e in enumerate(yy):
    e = list(e)
    e[1] = round(int(e[1])/div)
    yy[i] = e

In [66]:
yy.save()

In [86]:
yy = grasp.csv("insults_fr_final.csv")
# yy.clear()
# yy.update(x)
# yy.save()

In [95]:
pmi1 = grasp.csv("pmi_list_fr.csv")
selected = [e[0] for e in pmi1 if float(e[1]) > 0.13]
selected = set(selected)

In [96]:
insultes = []
for file in INSULT_FILES:
    if not file.startswith("cruci"):
        with open(file, encoding="utf8") as f:
            insultes.extend(f.read().split("\n"))
    else:
        with open(file, encoding="latin") as f:
            insultes.extend(f.read().split("\n"))
        
insultes = set(insultes)
selected = selected.union(insultes)
selected = list(selected)

In [183]:
new_ = []
for word, nbr in x:
    nrow = [""]*18
    nrow[8:] = ["FALSE"]*10
    nrow[1] = nbr
    nrow[3] = word
    new_.append(nrow)


In [98]:
x = [(k,v) for k,v in ch8_c.items() if k in selected]
x.sort(key=lambda x:x[1], reverse=True)